In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
import torch.nn.functional as F
from tqdm import tqdm  


class HateFakeDataset(Dataset):
    def __init__(self, texts, hate_labels, fake_labels, tokenizer, max_len=128):
        self.texts = texts
        self.hate_labels = hate_labels
        self.fake_labels = fake_labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        hate_label = self.hate_labels[idx]
        fake_label = self.fake_labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'hate_label': torch.tensor(hate_label, dtype=torch.long),
            'fake_label': torch.tensor(fake_label, dtype=torch.long)
        }

# Multi-task Model
class MultiTaskHateFakeClassifier(nn.Module):
    def __init__(self, bert_model, dropout_rate=0.3):
        super().__init__()
        self.bert = bert_model
        hidden_size = bert_model.config.hidden_size

        # Common layers
        self.dropout = nn.Dropout(dropout_rate)
        self.dense1 = nn.Linear(hidden_size, 512)
        self.dense2 = nn.Linear(512, 256)

        # Task-specific layers
        self.hate_classifier = nn.Linear(256, 2)
        self.fake_classifier = nn.Linear(256, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token

        x = self.dropout(pooled_output)
        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        x = F.relu(self.dense2(x))
        x = self.dropout(x)

        hate_output = F.softmax(self.hate_classifier(x), dim=1)
        fake_output = F.softmax(self.fake_classifier(x), dim=1)

        return hate_output, fake_output

# Training function 
def train_model(model, train_loader, val_loader, device, num_epochs=30, learning_rate=2e-5, patience=5):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    best_val_accuracy = 0
    early_stop_counter = 0
    best_epoch = 0
    best_model_wts = None

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            hate_labels = batch['hate_label'].to(device)
            fake_labels = batch['fake_label'].to(device)

            optimizer.zero_grad()
            hate_output, fake_output = model(input_ids, attention_mask)
            hate_loss = criterion(hate_output, hate_labels)
            fake_loss = criterion(fake_output, fake_labels)
            total_batch_loss = hate_loss + fake_loss

            total_batch_loss.backward()
            optimizer.step()
            total_loss += total_batch_loss.item()

        model.eval()
        val_loss = 0
        all_hate_preds, all_fake_preds = [], []
        all_hate_labels, all_fake_labels = [], []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating", leave=False):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                hate_labels = batch['hate_label'].to(device)
                fake_labels = batch['fake_label'].to(device)

                hate_output, fake_output = model(input_ids, attention_mask)
                hate_loss = criterion(hate_output, hate_labels)
                fake_loss = criterion(fake_output, fake_labels)
                val_loss += (hate_loss + fake_loss).item()

                all_hate_preds.extend(torch.argmax(hate_output, dim=1).cpu().numpy())
                all_fake_preds.extend(torch.argmax(fake_output, dim=1).cpu().numpy())
                all_hate_labels.extend(hate_labels.cpu().numpy())
                all_fake_labels.extend(fake_labels.cpu().numpy())

        # Calculate validation accuracy
        val_hate_accuracy = (torch.argmax(hate_output, dim=1) == hate_labels).float().mean().item()
        val_fake_accuracy = (torch.argmax(fake_output, dim=1) == fake_labels).float().mean().item()
        val_accuracy = (val_hate_accuracy + val_fake_accuracy) / 2

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Training Loss: {total_loss/len(train_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(val_loader):.4f}')
        print(f'Validation Accuracy: {val_accuracy:.4f}')

        print("Classification Report for Hate Speech Task:")
        print(classification_report(all_hate_labels, all_hate_preds))
        print("Classification Report for Fake News Task:")
        print(classification_report(all_fake_labels, all_fake_preds))

        # Check if this is the best epoch
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_epoch = epoch
            best_model_wts = model.state_dict()  

            early_stop_counter = 0
        else:
            early_stop_counter += 1

        # Early stopping check
        if early_stop_counter >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}.")
            break

    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'best_model.pt')  

    print(f"Training completed. Best epoch: {best_epoch+1} with validation accuracy: {best_val_accuracy:.4f}")
    print("Best Classification Report for Hate Speech Task:")
    print(classification_report(all_hate_labels, all_hate_preds))
    print("Best Classification Report for Fake News Task:")
    print(classification_report(all_fake_labels, all_fake_preds))


def main():
    # Load and preprocess your data
    df_train = pd.read_csv('A_trans.csv')  
    df_val = pd.read_csv('A_val_trans.csv')  

    # Clean and preprocess the labels
    for df in [df_train, df_val]:
        df.dropna(subset=['Hate', 'Fake', 'Tweet'], inplace=True)
        df['Hate'] = df['Hate'].astype(int)
        df['Fake'] = df['Fake'].astype(int)
        df = df[(df['Hate'].isin([0, 1])) & (df['Fake'].isin([0, 1]))]

    texts_train = df_train['Tweet'].values
    hate_labels_train = df_train['Hate'].values
    fake_labels_train = df_train['Fake'].values

    texts_val = df_val['Tweet'].values
    hate_labels_val = df_val['Hate'].values
    fake_labels_val = df_val['Fake'].values

    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
    bert_model = BertModel.from_pretrained('bert-base-multilingual-uncased')

    train_dataset = HateFakeDataset(texts_train, hate_labels_train, fake_labels_train, tokenizer)
    val_dataset = HateFakeDataset(texts_val, hate_labels_val, fake_labels_val, tokenizer)

    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = MultiTaskHateFakeClassifier(bert_model).to(device)

    train_model(model, train_loader, val_loader, device)

if __name__ == '__main__':
    main()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Epoch 26/30
Training Loss: 0.8445
Validation Loss: 1.0593
Classification Report for Hate Speech Task:
              precision    recall  f1-score   support

           0       0.70      0.67      0.69       286
           1       0.82      0.84      0.83       513

    accuracy                           0.78       799
   macro avg       0.76      0.76      0.76       799
weighted avg       0.78      0.78      0.78       799

Classification Report for Fake News Task:
              precision    recall  f1-score   support

           0       0.75      0.82      0.78       376
           1       0.82      0.75      0.79       423

    accuracy                           0.78       799
   macro avg       0.78      0.79      0.78       799
weighted avg       0.79      0.78      0.78       799



Epoch 27/30
Training Loss: 0.8422
Validation Loss: 1.0593
Classification Report for Hate Speech Task:
              precision    recall  f1-score   support

           0       0.71      0.63      0.67       286
           1       0.81      0.85      0.83       513

    accuracy                           0.77       799
   macro avg       0.76      0.74      0.75       799
weighted avg       0.77      0.77      0.77       799

Classification Report for Fake News Task:
              precision    recall  f1-score   support

           0       0.75      0.83      0.79       376
           1       0.83      0.75      0.79       423

    accuracy                           0.79       799
   macro avg       0.79      0.79      0.79       799
weighted avg       0.79      0.79      0.79       799



Epoch 28/30
Training Loss: 0.8408
Validation Loss: 1.0571
Classification Report for Hate Speech Task:
              precision    recall  f1-score   support

           0       0.68      0.70      0.69       286
           1       0.83      0.81      0.82       513

    accuracy                           0.77       799
   macro avg       0.75      0.76      0.75       799
weighted avg       0.77      0.77      0.77       799

Classification Report for Fake News Task:
              precision    recall  f1-score   support

           0       0.77      0.80      0.78       376
           1       0.81      0.78      0.80       423

    accuracy                           0.79       799
   macro avg       0.79      0.79      0.79       799
weighted avg       0.79      0.79      0.79       799



Epoch 29/30
Training Loss: 0.8372
Validation Loss: 1.0817
Classification Report for Hate Speech Task:
              precision    recall  f1-score   support

           0       0.68      0.68      0.68       286
           1       0.82      0.82      0.82       513

    accuracy                           0.77       799
   macro avg       0.75      0.75      0.75       799
weighted avg       0.77      0.77      0.77       799

Classification Report for Fake News Task:
              precision    recall  f1-score   support

           0       0.74      0.77      0.76       376
           1       0.79      0.76      0.78       423

    accuracy                           0.77       799
   macro avg       0.77      0.77      0.77       799
weighted avg       0.77      0.77      0.77       799



Epoch 30/30
Training Loss: 0.8319
Validation Loss: 1.0574
Classification Report for Hate Speech Task:
              precision    recall  f1-score   support

           0       0.70      0.67      0.69       286
           1       0.82      0.84      0.83       513

    accuracy                           0.78       799
   macro avg       0.76      0.76      0.76       799
weighted avg       0.78      0.78      0.78       799

Classification Report for Fake News Task:
              precision    recall  f1-score   support

           0       0.74      0.81      0.77       376
           1       0.81      0.75      0.78       423

    accuracy                           0.78       799
   macro avg       0.78      0.78      0.78       799
weighted avg       0.78      0.78      0.78       799

